In [1]:
from pprint import pprint

## Dataset

For this test, we will use the following tables R and S to write solutions to three parallel algorithms.


In [2]:
# R consists of 15 pairs, each comprising two attributes (nominal and numeric)


R = [
    ("Adele", 8),
    ("Bob", 22),
    ("Clement", 16),
    ("Dave", 23),
    ("Ed", 11),
    ("Fung", 25),
    ("Goel", 3),
    ("Harry", 17),
    ("Irene", 14),
    ("Joanna", 2),
    ("Kelly", 6),
    ("Lim", 20),
    ("Meng", 1),
    ("Noor", 5),
    ("Omar", 19),
]

# S consists of 8 pairs, each comprising two attributes (nominal and numeric)


S = [
    ("Arts", 8),
    ("Business", 15),
    ("CompSc", 2),
    ("Dance", 12),
    ("Engineering", 7),
    ("Finance", 21),
    ("Geology", 10),
    ("Health", 11),
    ("IT", 18),
]

## 1. Parallel Searching Algorithm (2 marks)

**This section consist of 4 sub-questions.**

In this task, you will build a **parallel search algorithm for range selection (continuous)** for a given query.

**Implement a parallel search algorithm** that uses local linear search (i.e., `linear_search()`) and is able to work with the hash partitioning method (i.e., `h_partition()`).
**Complete the code block between `### START CODE HERE ###` and `### END CODE HERE ###`.**


### 1.1 Local Search


In [3]:
# Linear search function


def linear_search(data, key):
    """
    Perform linear search on data for the given key.

    Arguments:
    data -- an input dataset (list OR numpy array)
    key -- an query attribute (number)

    Return:
    result --  list of matched records such as [('Adele', 8)]
                or the position of searched record
                whichever is more appropriate for your code
    """
    matches = []

    ### START CODE HERE ###

    # iterate over each record in input dataset

    for record in data:

        # check if number attribute of record matches key

        if record[1] == key:

            # match found - append the record to the matches list

            matches.append(record)
    ### END CODE HERE ###

    return matches

In [4]:
results = linear_search(R, 20)
print(results)

[('Lim', 20)]


### 1.2 Hash Partition


In [5]:
# Define a simple hash function.


def s_hash(x, n):
    """
    Define a simple hash function for demonstration

    Arguments:
    x -- an input record attribute (int)
    n -- the number of processors (int)

    Return:
    result -- the hash value of x
    """
    return x % n

In [6]:
# Hash data partitioning function.
# We will use the s_hash function defined above


def h_partition(data, n):
    """
    Perform hash data partitioning on data

    Arguments:
    data -- an input dataset (list)
    n -- the number of processors (int)

    Return:
    result -- the partitioned subsets of D
    """
    partitions = {}

    ### START CODE HERE ###

    # iterate the logic for each data record

    for record in data:

        # get the hash key of the input

        hash_key = s_hash(record[1], n)

        # if the key exists

        if hash_key in partitions.keys():

            # get the value list of the key

            partition_records = partitions[hash_key]

            # add the new input to the list

            partition_records.append(record)

            # update the key with the new value list

            partitions[hash_key] = partition_records
        # if the key does not exist

        else:

            # create an empty value list

            partition_records = list()

            # add the input to the list

            partition_records.append(record)

            # add the value list to the key

            partitions[hash_key] = partition_records
    ### END CODE HERE ###

    return partitions

In [7]:
partitions = h_partition(R, 5)
pprint(partitions)

{0: [('Fung', 25), ('Lim', 20), ('Noor', 5)],
 1: [('Clement', 16), ('Ed', 11), ('Kelly', 6), ('Meng', 1)],
 2: [('Bob', 22), ('Harry', 17), ('Joanna', 2)],
 3: [('Adele', 8), ('Dave', 23), ('Goel', 3)],
 4: [('Irene', 14), ('Omar', 19)]}


### 1.3 Parallel Search


In [8]:
from multiprocessing import Pool


def collect_result(results, result):
    # Callback for when linear_search returns a result.
    # The results list is modified only by the main process, not the pool workers.

    results.append(result)


# Parallel searching algorithm for range selection


def parallel_search_range(data, query_range, n_processor):
    """
    Perform parallel search for range selection on data for the given key.

    Arguments:
    data -- the input dataset (list)
    query_range -- a tuple with inclusive range end-points (e.g. [30, 50] means the interval 30-50)
    n_processor -- the number of parallel processors (int)

    Return:
    results -- the matched record information
    """

    pool = Pool(processes=n_processor)

    ### START CODE HERE ###

    # create an empty list to store the results

    results = []

    # partition the data using hash function

    partitioned_data = h_partition(data, n_processor)

    # iterate over the query range

    for query in range(query_range[0], query_range[1] + 1):

        # get the hash key of the query

        query_hash = s_hash(query, n_processor)

        # get the data records of the hash key and convert it to a list

        range_data = list(partitioned_data[query_hash])

        # apply linear search on the data records

        result = pool.apply_async(linear_search, [range_data, query])

        # add an if condition because its appending regardless, so there are eempty lists

        if result.get():
            # append the result to the results list

            results.append(result.get())
    # close the pool

    pool.close()

    # wait for the worker processes to terminate

    pool.join()

    ### END CODE HERE ###

    return results

In [9]:
n_processor = 3
results = parallel_search_range(R, [5, 15], n_processor)
pprint(results)

[[('Noor', 5)], [('Kelly', 6)], [('Adele', 8)], [('Ed', 11)], [('Irene', 14)]]


### 1.4 Parallel Search Variants


**Briefly answer the following.**

1. How would the parallel search you implemented need to be changed if we switched to round-robin partition for the data? Provide a small snippet of code with the modified search loop.


**Your Answers**:

To switch to using round-robin partitioning instead of hash partitioning, we would need to:

1. Replace the function call `h_partition()` with `rr_partition()` as this is the round-robin partitioning function. The `rr_partition()` function would divide the data in each record in turn as it is allocated to a processing element in a clockwise manner. For example, it would divide the first record of a table to be partitioned is distributed to the first processing element, the second record to the second processing element, and so on.

2. The modified search loop would iterate over each processor's sublist of data, submit a parallel linear search task for the query on each processor's data subset using `pool.apply_async()`, and collect the results from each processor, appending them to the final results list.

```python
partitioned_data = rr_partition(data, n_processor)

for query in range(query_range[0], query_range[1] + 1):
    results_per_processor = []
    for processor_data in partitioned_data:
        result = pool.apply_async(linear_search, [processor_data, query])
        results_per_processor.append(result)

    for result in results_per_processor:
        if result.get():
            results.append(result.get())
```

Using `rr_partition()` divides it among the processors and hence, for each query value, we iterate over each processor's sublist of data, submitting a parallel linear search task for the query on each processor's data subset using `pool.apply_async()` and store the result objects in `results_per_processor`. After submitting the tasks for all processors, we iterate over the result objects, retrieve the actual results using `result.get()`, and append the non-empty results to the final `results` list.

Using round-robin partitioning ensures that the data is distributed evenly across the processors. However, the search operation needs to be performed on each processor's data subset for every query value. This may be less efficient compared to using hash partitioning, where the search can be limited to a specific processor based on the hash value of the query. Round-robin partitioning guarantees better load balancing among the processors, as each processor receives an equal amount of data to process.


## 2. Parallel Join Algorithm (4 marks)

**This section consist of 5 sub-questions.**

In this task, you will implement a **disjoint-partitioning based parallel join algorithm**. This algorithm consist of two stages.

1. Data disjoint-partitioning across processors.
1. Local join in each processor.

As a data partitioning method, use the range partitioning algorithm (i.e. `range_partition( )`). Assume that we have **3 parallel processors**, processor 1 will get records with join attribute value between 1 and 9, processor 2 between 10 and 19, and processor 3 between 20 and 29. Note that both tables R and S need to be partitioned using the same ranges described earlier.

As a joining technique, use the hash based join algorithm (i.e., `HB_join( )`). **Complete the code block between `### START CODE HERE ###` and `### END CODE HERE ###`.**


### 2.1 Range Partition


In [10]:
# Range data partitionining function (Need to modify as instructed above)


def range_partition(data, range_indices):
    """
    Perform range data partitioning on data based on the join attribute

    Arguments:
    data -- an input dataset (list)
    range_indices -- the range end-points (e.g., [5, 10, 15] means
                        4 intervals: x < 5, 5 <= x < 10, 10 <= x < 15, 15 <= x)

    Return:
    result -- the partitioned subsets of D
    """
    result = []

    ### START CODE HERE ###

    # copy the data to a new list

    partitioned_data = list(data)

    # sort the data based on the join attribute

    partitioned_data.sort(key=lambda x: x[1])

    # get the number of bins

    n_bins = len(range_indices)

    # iterate over the range indices

    for i in range(n_bins):

        # get the data records within the current range

        s = [x for x in partitioned_data if x[1] < range_indices[i]]

        # add the data records to the result list

        result.append(s)

        # get the last element of the current range

        last_element = s[len(s) - 1]

        # get the index of the last element

        last_index = partitioned_data.index(last_element)

        # update the data records

        partitioned_data = partitioned_data[int(last_index) + 1 :]
    # add the remaining data records to the result list

    result.append([x for x in partitioned_data if x[1] >= range_indices[n_bins - 1]])

    ### END CODE HERE ###

    return result

In [11]:
pprint(range_partition(R, [10, 20]))

[[('Meng', 1),
  ('Joanna', 2),
  ('Goel', 3),
  ('Noor', 5),
  ('Kelly', 6),
  ('Adele', 8)],
 [('Ed', 11), ('Irene', 14), ('Clement', 16), ('Harry', 17), ('Omar', 19)],
 [('Lim', 20), ('Bob', 22), ('Dave', 23), ('Fung', 25)]]


### 2.2 Hash Join


In [12]:
def H(r):
    """
    We define a hash function 'H' that is used in the hashing process works
    by summing the first and second digits of the hashed attribute, which
    in this case is the join attribute.

    Arguments:
    r -- a record where hashing will be applied on its join attribute

    Return:
    result -- the hash index of the record r
    """

    # Convert the value of the join attribute into the digits

    digits = [int(d) for d in str(r[1])]

    # Calulate the sum of elemenets in the digits

    return sum(digits)

In [13]:
def HB_join(T1, T2):
    """
    Perform the hash-based join algorithm.
    The join attribute is the numeric attribute (in second position) in the input tables T1 & T2.

    Arguments:
    T1 & T2 -- Tables to be joined

    Return:
    result -- the joined table
    """
    result = []
    dic = {}  # We will use a dictionary

    # For each record in table T2

    for s in T2:
        # Hash the record based on join attribute value using hash function H into hash table

        s_key = H(s)
        if s_key in dic:
            dic[s_key].add(s)  # If there is an entry
        else:
            dic[s_key] = {s}
    print(dic)
    # For each record in table T1 (probing)

    for r in T1:
        # Hash the record based on join attribute value using H

        r_key = H(r)
        # If an index entry is found Then

        if r_key in dic:
            # Compare each record on this index entry with the record of table T1

            for item in dic[r_key]:
                if item[1] == r[1]:
                    # Put the rsult

                    result.append((r[0], r[1], item[0]))
    return result

In [14]:
# print the partitioned result

HB_join(R, S)

{8: {('Arts', 8)}, 6: {('Business', 15)}, 2: {('Health', 11), ('CompSc', 2)}, 3: {('Finance', 21), ('Dance', 12)}, 7: {('Engineering', 7)}, 1: {('Geology', 10)}, 9: {('IT', 18)}}


[('Adele', 8, 'Arts'), ('Ed', 11, 'Health'), ('Joanna', 2, 'CompSc')]

### 2.3 Parallel Join


In [15]:
# Include this package for parallel processing

import multiprocessing as mp


def consolidate_result(results, result):
    # This is called whenever HB_Join(T1, T2) returns a result.
    # The results list is modified only by the main process, not the pool workers.

    results.append(result)


def DPBP_join(T1, T2, n_processor):
    """
    Perform a disjoint partitioning-based parallel join algorithm.
    The join attribute is the numeric attribute in the input tables T1 & T2

    Arguments:
    T1 & T2 -- Tables to be joined
    n_processor -- the number of parallel processors

    Return:
    result -- the joined table
    """

    ### START CODE HERE ###

    results = []

    result = []

    # partition t1 and t2 using range partitioning

    T1_subsets = range_partition(T1, [10, 20])

    T2_subsets = range_partition(T2, [10, 20])

    # apply local join for each processor

    pool = mp.Pool(processes=n_processor)

    # iterate over the subsets

    for i in range(len(T1_subsets)):

        # perform hb_join on partitioned subsets

        result = pool.apply_async(HB_join, [T1_subsets[i], T2_subsets[i]])

        # append the result to the results list

        output = result.get()

        # append the result

        results.append(output)
    ### END CODE HERE ###

    return results

In [16]:
n_processor = 3
DPBP_join(R, S, n_processor)

{2: {('CompSc', 2)}, 7: {('Engineering', 7)}, 8: {('Arts', 8)}}
{1: {('Geology', 10)}, 2: {('Health', 11)}, 3: {('Dance', 12)}, 6: {('Business', 15)}, 9: {('IT', 18)}}
{3: {('Finance', 21)}}


[[('Joanna', 2, 'CompSc'), ('Adele', 8, 'Arts')], [('Ed', 11, 'Health')], []]

### 2.4 Duplication Outer Join Algorithm (DOJA)

In this task, you will implement a **Duplication Outer Join Algorithm (DOJA)**. This algorithm consist of four main steps.

1. (Replication): Duplicate the small table
2. (Local Inner Join): Perform inner join in each processor
3. (Distribution): Reshuffle the inner join result based on R.x
4. (Local Outer Join): Perform outer join between the initial table R and the inner join result

Assume **n_processor=3** and the dataset is initially partition using the **hash** method. For joining technique, please use the provided **outer join** function for the inner join and outer join operation. **Complete the code block between `### START CODE HERE ###` and `### END CODE HERE ###`.**


In [17]:
# R consists of 15 pairs, each comprising two attributes (nominal and numeric)

R = [
    ("Adele", 8),
    ("Bob", 22),
    ("Clement", 16),
    ("Dave", 23),
    ("Ed", 11),
    ("Fung", 25),
    ("Goel", 3),
    ("Harry", 17),
    ("Irene", 14),
    ("Joanna", 2),
    ("Kelly", 6),
    ("Lim", 20),
    ("Meng", 1),
    ("Noor", 5),
    ("Omar", 19),
]

# S consists of 8 pairs, each comprising two attributes (nominal and numeric)

S = [
    ("Arts", 8),
    ("Business", 15),
    ("CompSc", 2),
    ("Dance", 12),
    ("Engineering", 7),
    ("Finance", 21),
    ("Geology", 10),
    ("Health", 11),
    ("IT", 18),
]

In [18]:
import numpy as np


def H(r):
    """
    We define a hash function 'H' that is used in the hashing process works
    by summing the first and second digits of the hashed attribute, which
    in this case is the join attribute.

    Arguments:
    r -- a record where hashing will be applied on its join attribute

    Return:
    result -- the hash index of the record r
    """

    # Convert the value of the join attribute into the digits

    digits = [int(d) for d in str(r[1])]

    # Calulate the sum of elemenets in the digits

    return sum(digits)


def outer_join(L, R, join="left"):
    """outer join using Hash-based join algorithm"""

    if join == "right":
        L, R = R, L
    # inner join

    if join == "inner":
        h_dic = {}
        for r in R:
            h_r = H(r)
            if h_r in h_dic.keys():
                h_dic[h_r].add(r)
            else:
                h_dic[h_r] = {r}
        result = []
        for l in L:
            h_l = H(l)
            if h_l in h_dic.keys():
                for item in h_dic[h_l]:
                    if item[1] == l[1]:
                        result.append([l[0], item[1], item[0]])
        return result
    elif join in ["left", "right"]:
        h_dic = {}
        for r in R:
            print(r)
            h_r = H(r)
            if h_r in h_dic.keys():
                h_dic[h_r].add(r)
            else:
                h_dic[h_r] = {r}
        print(h_dic)

        result = []
        for l in L:
            isFound = False  # to check whether there is a match found.
            h_l = H(l)

            if h_l in h_dic.keys():
                for item in h_dic[h_l]:
                    if item[1] == l[1]:
                        result.append([l[0], item[1], item[2]])
                        isFound = True
                        break
            if not isFound:
                result.append([l[0], l[1], str(np.nan)])
        return result
    else:
        raise AttributeError("join should be in {left, right, inner}.")

In [19]:
def hash_distribution(T, n):
    """distribute data using hash partitioning"""

    # Define a simple hash function for demonstration

    def s_hash(x, n):
        h = x % n
        return h

    result = {}
    for t in T:
        h_key = s_hash(t[1], n)
        if h_key in result.keys():
            result[h_key].add(tuple(t))
        else:
            result[h_key] = {tuple(t)}
    return result

In [20]:
import multiprocessing as mp


def DOJA(L, R, n):
    """left outer join using DOJA"""

    ### Start CODE

    # step 1: determine the smaller relation and broadcast it to all processors

    if len(L) < len(R):

        # broadcast L to all processors

        broadcasted_table = [tuple(s) for s in L]

        # R becomes the larger table

        L = [tuple(r) for r in R]
    else:

        # broadcast R to all processors

        broadcasted_table = [tuple(s) for s in R]

        # L remains the larger table

        L = [tuple(l) for l in L]
    # create a dictionary to store the broadcasted relation

    broadcasted_dict = {i: broadcasted_table for i in range(n)}

    # step 2: perform local inner join on each processor

    local_inner_join_results = {}

    # hash distribution of the larger table L

    partitioned_L = hash_distribution(L, n)

    for processor_id in range(n):

        # inner join on each processor

        local_inner_join_results[processor_id] = outer_join(
            partitioned_L[processor_id], broadcasted_dict[processor_id], join="inner"
        )
    # step 3: redistribute inner join results based on the join attribute from the larger relation

    redistributed_inner_join_results = {}

    for processor_id in range(n):

        for inner_join_tuple in local_inner_join_results[processor_id]:

            # apply hash function on the join attribute of the tuple

            hash_value = H((None, inner_join_tuple[1]))

            if hash_value in redistributed_inner_join_results:

                redistributed_inner_join_results[hash_value].append(inner_join_tuple)
            else:

                redistributed_inner_join_results[hash_value] = [inner_join_tuple]
    # step 4: perform local left outer join

    final_output = {}

    for processor_id in range(n):

        final_output[processor_id] = []

        # create a set of join attributes from broadcasted_table that found a match

        matched_join_attributes = {
            join_tuple[1] for join_tuple in local_inner_join_results[processor_id]
        }

        # include tuples from L that either match with broadcasted_table or are appended with np.nan

        for L_tuple in partitioned_L[processor_id]:

            if L_tuple[1] in matched_join_attributes:

                # if a match was found, keep the matched tuple from local_inner_join_results

                matched_tuple = next(
                    (
                        join_tuple
                        for join_tuple in local_inner_join_results[processor_id]
                        if join_tuple[1] == L_tuple[1]
                    ),
                    None,
                )

                final_output[processor_id].append(list(matched_tuple))
            else:

                # if no match was found, append np.nan

                final_output[processor_id].append(list(L_tuple) + [np.nan])
    ### End CODE

    return final_output

In [21]:
DOJA(R, S, 3)

{0: [['Kelly', 6, nan], ['Goel', 3, nan]],
 1: [['Fung', 25, nan],
  ['Omar', 19, nan],
  ['Meng', 1, nan],
  ['Clement', 16, nan],
  ['Bob', 22, nan]],
 2: [['Joanna', 2, 'CompSc'],
  ['Lim', 20, nan],
  ['Adele', 8, 'Arts'],
  ['Ed', 11, 'Health'],
  ['Noor', 5, nan],
  ['Harry', 17, nan],
  ['Irene', 14, nan],
  ['Dave', 23, nan]]}

### 2.5 Parallel Join Variants


Briefly answer the following question.

1. For each partitioning algorithm besides range-partitioning, state and justify whether we can use it with the code for disjoint-partitioning based parallel join above.


**Your answer**:

1. Round-robin data partitioning: Round-robin partitioning can be used with disjoint-partitioning based parallel join. It would require replacing the `range_partition` function with `rr_partition` function that allocates records to processing elements equally in clockwise manner. The join condition in the `HB_join` function would need to be modified to handle the non-partitioned join attributes. Round-robin partitioning is suitable when the join attributes are not known in advance or not used for partitioning.

2. Hash data partitioning: Hash partitioning can be used with the given code by replacing the `range_partition` function with the `h_partition` function that applies a hash function to a specific attribute. Also, the `HB_join` function would need to be modified to use the same hash function for joining the partitions. Hash partitioning is suitable for equi-joins on the hashed attribute, as it groups records with the same hash value into the same partition.

3. Random-unequal data partitioning: Random-unequal partitioning is not suitable as it assumes disjoint partitions based on a specific attribute. Random-unequal partitioning randomly allocates records to partitions, resulting in uneven distribution and potential data skew, which can affect the performance of the parallel join algorithm.

4. Hybrid-Range Partitioning Strategy (HRPS): HRPS can be used with the given code by replacing the `range_partition` function with an `hrps_partition` function. HRPS combines range partitioning with hash and round-robin partitioning, partitioning records into small fragments based on a range of attribute values and distributing the fragments among processors in a round-robin fashion.

5. MAGIC (Multiattribute Grid Declustering): MAGIC is not directly compatible as it assumes single-attribute partitioning. MAGIC partitions records based on multiple attributes using a grid structure, supporting both range and exact match queries on the partitioning attributes. The join operation would need to consider the grid structure and the multiple attributes for partitioning and joining.

6. BERD (Bubba's Extended Range Declustering): BERD is not suitable as it alsoa assumes single-attribute partitioning. BERD uses two levels of partitioning: primary (range) and secondary (range on another attribute), and constructs an auxiliary table for the secondary attribute. Modifying the partitioning logic to handle the two-level partitioning and the auxiliary table would be necessary. The join operation would also need to consider the secondary attribute for partitioning and joining.


## 3. Parallel Sorting Algorithm (2 marks)

In this task, you will implement **parallel binary-merge sort** method. It has two phases same as the parallel merge-all sort that you learnt in the labs.

1. Local sort
2. Final merge.

The first phase is similar to the parallel merge-all sort. The second phase, the merging phase, is pipelined instead of concentrating on one processor. In this phase, we take the results from two processors and then merge the two in one processor, called binary merging. The result of the merge between two processors is passed on to the next level until one processor (the host) is left.

**Complete the code block between `### START CODE HERE ###` and `### END CODE HERE ###`.**
Assume that we use the round robin partitioning method (i.e. `rr_partition()`).


### 3.1 Round-robin Partition


In [22]:
# round-robin data partitioning function


def rr_partition(data, n):
    """
    Perform data partitioning on data

    Arguments:
    data -- an input dataset which is a list
    n -- the number of processors

    Return:
    result -- the paritioned subsets of D
    """
    result = []
    for i in range(n):
        result.append([])
    ### START CODE HERE ###

    # iterate over each data point in the input dataset

    for index, element in enumerate(data):

        # calculate the index of the partition to place the current data point

        index_bin = (int)(index % n)

        # append the current data point to the corresponding partition

        result[index_bin].append(element)
    ### END CODE HERE ###

    return result

In [23]:
# Test the round-robin partitioning function

result = rr_partition(R, 3)
pprint(result)

[[('Adele', 8), ('Dave', 23), ('Goel', 3), ('Joanna', 2), ('Meng', 1)],
 [('Bob', 22), ('Ed', 11), ('Harry', 17), ('Kelly', 6), ('Noor', 5)],
 [('Clement', 16), ('Fung', 25), ('Irene', 14), ('Lim', 20), ('Omar', 19)]]


### 3.2 Serial Sort


In [24]:
def qsort(arr):
    """
    Quicksort a list

    Arguments:
    arr -- the input list to be sorted

    Return:
    result -- the sorted arr
    """
    if len(arr) <= 1:
        return arr
    else:
        # take the first element as the pivot

        pivot = arr[0]
        left_arr = [x for x in arr[1:] if x[1] < pivot[1]]
        right_arr = [x for x in arr[1:] if x[1] >= pivot[1]]
        # uncomment this to see what to print
        # print("Left:" + str(left_arr)+" Pivot : "+ str(pivot)+" Right: " + str(right_arr))

        sorted_arr = qsort(left_arr) + [pivot] + qsort(right_arr)

        return sorted_arr

In [25]:
qsort(R)

[('Meng', 1),
 ('Joanna', 2),
 ('Goel', 3),
 ('Noor', 5),
 ('Kelly', 6),
 ('Adele', 8),
 ('Ed', 11),
 ('Irene', 14),
 ('Clement', 16),
 ('Harry', 17),
 ('Omar', 19),
 ('Lim', 20),
 ('Bob', 22),
 ('Dave', 23),
 ('Fung', 25)]

In [26]:
# Let's first look at 'k-way merging algorithm' that will be used
# to merge sub-record sets in our external sorting algorithm.

import sys


# Find the smallest record


def find_min(records):
    """
    Find the smallest record

    Arguments:
    records -- the input record set

    Return:
    result -- the smallest record's index
    """
    m = records[0]
    index = 0
    for i in range(len(records)):
        if records[i][1] < m[1]:
            index = i
            m = records[i]
    return index


def k_way_merge(record_sets):
    """
    K-way merging algorithm

    Arguments:
    record_sets -- the set of multiple sorted sub-record sets

    Return:
    result -- the sorted and merged record set
    """
    # indexes will keep the indexes of sorted records in the given buffers

    indexes = []
    for _ in record_sets:
        indexes.append(0)  # initialisation with 0
    # final result will be stored in this variable

    result = []
    while True:
        merged_result = []  # the merging unit (i.e. # of the given buffers)

        # This loop gets the current position of every buffer

        for i in range(len(record_sets)):
            if indexes[i] >= len(record_sets[i]):
                merged_result.append((None, sys.maxsize))
            else:
                merged_result.append(record_sets[i][indexes[i]])
        # find the smallest record

        smallest = find_min(merged_result)

        # if we only have sys.maxsize on the tuple, we reached the end of every record set

        if merged_result[smallest][1] == sys.maxsize:
            break
        # This record is the next on the merged list

        result.append(record_sets[smallest][indexes[smallest]])
        indexes[smallest] += 1
    return result

In [27]:
# Test k-way merging method

buffers = rr_partition(R, 3)
print(buffers)
result = k_way_merge([qsort(b) for b in buffers])
pprint(result)

[[('Adele', 8), ('Dave', 23), ('Goel', 3), ('Joanna', 2), ('Meng', 1)], [('Bob', 22), ('Ed', 11), ('Harry', 17), ('Kelly', 6), ('Noor', 5)], [('Clement', 16), ('Fung', 25), ('Irene', 14), ('Lim', 20), ('Omar', 19)]]
[('Meng', 1),
 ('Joanna', 2),
 ('Goel', 3),
 ('Noor', 5),
 ('Kelly', 6),
 ('Adele', 8),
 ('Ed', 11),
 ('Irene', 14),
 ('Clement', 16),
 ('Harry', 17),
 ('Omar', 19),
 ('Lim', 20),
 ('Bob', 22),
 ('Dave', 23),
 ('Fung', 25)]


In [28]:
def serial_sorting(dataset, buffer_size):
    """
    perform a serial external sorting method based on sort-merge
    the buffer size determines the size of each sub-record set

    arguments:
    dataset -- the entire record set to be sorted
    buffer_size -- the buffer size determining the size of each sub-record set

    return:
    result -- the sorted record set
    """

    if buffer_size <= 2:
        print("Error: buffer size should be greater than 2")
        return

    result = []

    ### START CODE HERE ###

    sorted_set = []

    # sort phase

    start_pos = 0
    N = len(dataset)

    while start_pos < N:

        # read B-records from the input, where B = buffer_size
        subset = dataset[start_pos : start_pos + buffer_size]

        # sort the subset using quicksort
        sorted_subset = qsort(subset)

        # append the sorted subset to the sorted set
        sorted_set.append(sorted_subset)

        # update the starting position to the next position
        start_pos += buffer_size

    # merge phase

    # merge buffer size is one less than the buffer size to ensure that the memory is enough to store the merged records
    merge_buffer_size = buffer_size - 1
    dataset = sorted_set

    # iterate until the dataset is completely merged
    while len(dataset) > 1:

        merged_set = []
        N = len(dataset)
        start_pos = 0

        while start_pos < N:

            # read c-record sets from merged record, where C = merge_buffer_size
            subset = dataset[start_pos : start_pos + merge_buffer_size]

            # merge the subset using k-way merge
            merged_set.append(k_way_merge(subset))

            # update the starting position to the next position
            start_pos += merge_buffer_size

        # update the dataset with the merged set
        dataset = merged_set

    # the final result is the first (and only) element in the dataset
    result = dataset[0]

    ### END CODE HERE ###

    return result

In [29]:
print(R)

[('Adele', 8), ('Bob', 22), ('Clement', 16), ('Dave', 23), ('Ed', 11), ('Fung', 25), ('Goel', 3), ('Harry', 17), ('Irene', 14), ('Joanna', 2), ('Kelly', 6), ('Lim', 20), ('Meng', 1), ('Noor', 5), ('Omar', 19)]


In [30]:
result = serial_sorting(R, 4)
print("final sorting result:" + str(result))

final sorting result:[('Meng', 1), ('Joanna', 2), ('Goel', 3), ('Noor', 5), ('Kelly', 6), ('Adele', 8), ('Ed', 11), ('Irene', 14), ('Clement', 16), ('Harry', 17), ('Omar', 19), ('Lim', 20), ('Bob', 22), ('Dave', 23), ('Fung', 25)]


### 3.3 Parallel Binary-merge Sort


In [31]:
# Include this package for parallel processing


import multiprocessing as mp


def parallel_binary_merge_sorting(dataset, n_processor, buffer_size):
    """
    Perform a parallel binary-merge sorting method

    Arguments:
    dataset -- entire record set to be sorted
    n_processor -- number of parallel processors
    buffer_size -- buffer size determining the size of each sub-record set

    Return:
    result -- the merged record set
    """

    if buffer_size <= 2:
        print("Error: buffer size should be greater than 2")
        return
    result = []

    ### START CODE HERE ###

    # partition using round-robin partitioning

    data_sets = rr_partition(dataset, n_processor)

    # sort phase

    pool = mp.Pool(processes=n_processor)

    sorted_lists = []
    parallel_result = []

    # apply serial sorting on each processor

    for i, data_set in enumerate(data_sets):
        parallel_result.append(
            pool.apply_async(serial_sorting, [data_set, buffer_size])
        )
    # get the sorted sub-record sets

    for i, process in enumerate(parallel_result):
        sorted_list = process.get()
        sorted_lists.append(sorted_list)
    pool.close()

    # merge phase

    iteration = 1

    # merge the sorted sub-record sets using k-way merge

    while len(sorted_lists) > 1:

        # calculate the number of merges

        num_merges = len(sorted_lists) // 2

        # check if the number of sorted sub-record sets is odd

        odd_list = None

        # if the number of sorted sub-record sets is odd, pop the last one to make it even

        if len(sorted_lists) % 2 != 0:
            odd_list = sorted_lists.pop()
        pool = mp.Pool(processes=num_merges)
        merge_tasks = []

        # apply k-way merge on each processor

        for i in range(num_merges):
            merge_tasks.append(
                pool.apply_async(
                    k_way_merge, ([sorted_lists[i * 2], sorted_lists[i * 2 + 1]],)
                )
            )
        # get the merged sub-record sets

        merged_lists = [task.get() for task in merge_tasks]
        pool.close()
        pool.join()

        # append the odd list if it exists

        if odd_list:
            merged_lists.append(odd_list)
        # update the sorted sub-record sets

        sorted_lists = merged_lists
        iteration += 1
    result = sorted_lists[0]

    ### END CODE HERE ###

    return result

In [32]:
result = parallel_binary_merge_sorting(R, 10, 20)
print("final result:" + str(result))

final result:[('Meng', 1), ('Joanna', 2), ('Goel', 3), ('Noor', 5), ('Kelly', 6), ('Adele', 8), ('Ed', 11), ('Irene', 14), ('Clement', 16), ('Harry', 17), ('Omar', 19), ('Lim', 20), ('Bob', 22), ('Dave', 23), ('Fung', 25)]


### 3.4 Parallel Binary-merge Behavior


Briefly answer the following.

1. Does parallel binary-merge utilize all available processors? State and justify with respect to each phase: sort, merge.


**Your answer**:

Parallel binary-merge sort does not fully utilize all available processors throughout the entire sorting process.

**Sort:**

In the sort phase, the input data is first partitioned across all processors using a round-robin scheme, ensuring an even distribution of data among the processors. Each processor then independently performs a serial external sort on its local data partition. Local sorting in each processor is carried out using a normal serial external sorting mechanism, as the data is assumed to be too large to fit in main memory. During the local sort phase, all processors are actively sorting their respective data partitions simultaneously. There is no idle time or underutilization of processors. Therefore, the local sort phase achieves excellent utilization of all available processors, as they are all working in parallel to sort their assigned data partitions.

**Merge:**

The merging phase in parallel binary-merge sort follows a pipelined, hierarchical approach. The merging process continues iteratively until only one sorted list remains. In each iteration, the number of sorted sublists is halved by merging pairs of sublists. If the number of sublists is odd, the last sublist is kept aside and merged in the next iteration. The merging tasks are distributed among a number of processors equal to half the number of sublists. Each processor merges a pair of sublists using the k_way_merge function. However, as the merging progresses up the binary merge tree, the number of active processors decreases by half at each level. In the final merge step, only a single processor remains to merge the last two sorted sublists. Throughout the merge phase, not all processors are actively utilized. At each level of the merge tree, half of the processors from the previous level become idle. Moreover, there is still no true parallelism in the merging because only a subset, not all, of the available processors are used.

In summary, parallel binary-merge sort achieves excellent processor utilization during the local sort phase, where all processors are actively sorting their respective data partitions in parallel. However, during the merge phase, processor utilization decreases as the merging progresses up the binary merge tree. At each level of the merge tree, half of the processors become idle. The final merge step is performed by a single processor. So while the pipelined merging in parallel binary-merge sort improves upon the single-processor merging in parallel merge-all sort by distributing the merging workload among multiple processors, it still fails to achieve optimal utilization of all available processors throughout the entire merging process.

Parallel binary-merge sort does not fully utilize all available processors, particularly during the merge phase, due to the hierarchical binary merge approach that results in a decreasing number of active processors at each level of the merge tree.


## 4 Parallel GroupBy (2 marks)


### 4.1 Parallel GroupBy with Merge-All

In this task, you will implement **Parallel GroupBy with Merge-All** method. It invloves the following two steps:

1. Local aggregate in each processor
2. Global aggregation

Assume **n_processor=4** and the dataset has already been pre-partitioned. For local aggregation, please use the provided **local_groupby** function. **Complete the code block between `### START CODE HERE ###` and `### END CODE HERE ###`.**


In [33]:
D1 = [("A", 1), ("B", 2), ("C", 3), ("A", 10), ("B", 20), ("C", 30)]
D2 = [("A", 2), ("B", 3), ("C", 4), ("A", 20), ("B", 30), ("C", 40)]
D3 = [("A", 3), ("B", 4), ("C", 5), ("A", 30), ("B", 40), ("C", 50)]
D4 = [("A", 4), ("B", 5), ("C", 6), ("A", 40), ("B", 50), ("C", 60)]

In [34]:
# The first step in the merge-all groupby method


def local_groupby(dataset):
    """
    Perform a local groupby method

    Arguments:
    dataset -- entire record set to be merged

    Return:
    result -- the aggregated record set according to the group_by attribute index
    """

    dict = {}
    for index, record in enumerate(dataset):
        key = record[0]
        val = record[1]
        if key not in dict:
            dict[key] = 0
        dict[key] += val
    return dict

In [35]:
result = local_groupby(D1)
print(result)

{'A': 11, 'B': 22, 'C': 33}


In [36]:
import multiprocessing as mp


def parallel_merge_all_groupby(dataset):
    """
    Perform a parallel merge_all groupby method

    Arguments:
    dataset -- entire record set to be merged

    Return:
    result -- the aggregated record dictionary according to the group_by attribute index
    """

    result = {}

    ### START CODE HERE ###

    # number of processors is equal to the number of records in the dataset

    n_processor = len(dataset)
    pool = mp.Pool(processes=n_processor)

    #  local aggregation

    local_result = []

    # apply local_groupby on each processor

    for s in dataset:

        # apply local_groupby on each processor

        local_result.append(pool.apply(local_groupby, [s]))
    pool.close()

    # global aggregation

    # iterate over the local results

    for r in local_result:

        for key, val in r.items():

            # if the key exists

            if key not in result:

                # add the key to the result

                result[key] = 0
            # add the value to the existing key

            result[key] += val
    ### END CODE HERE ###

    return result

In [37]:
E = [D1, D2, D3, D4]
result = parallel_merge_all_groupby(E)
print(result)

{'A': 110, 'B': 154, 'C': 198}


### 4.2 Redistribution Method

In this task, you will implement **Parallel GroupBy with Redistribution Method** method. It invloves the following two steps:

1. (Partitioning phase): Redistribute raw records to all processor
2. (Aggregation phase): Each processor performs a local aggregation

Assume **n_processor=4** and the dataset should be partitioned into 4 groups using **range partition** Where first group with A & B, second with C & D, third with E & F and fourth with G & H. For local aggregation, please use the provided **local_groupby** function. **Complete the code block between `### START CODE HERE ###` and `### END CODE HERE ###`.**


In [38]:
D = [
    ("A", 1),
    ("B", 2),
    ("C", 3),
    ("D", 10),
    ("E", 20),
    ("F", 30),
    ("G", 20),
    ("H", 30),
    ("A", 2),
    ("B", 3),
    ("C", 4),
    ("D", 20),
    ("E", 30),
    ("F", 40),
    ("G", 30),
    ("H", 40),
    ("A", 3),
    ("B", 4),
    ("C", 5),
    ("D", 30),
    ("E", 40),
    ("F", 50),
    ("G", 40),
    ("H", 50),
    ("A", 4),
    ("B", 5),
    ("C", 6),
    ("D", 40),
    ("E", 50),
    ("F", 60),
    ("G", 50),
    ("H", 60),
]

#### Create range partition function based on GroupBy attribute


In [39]:
# Range data partitionining function (Need to modify as instructed above)


def range_partition(data, range_indices):
    """
    Perform range data partitioning on data based on the join attribute

    Arguments:
    data -- an input dataset (list of tuple)
    range_indices -- the range end-points (e.g., ['C', 'E', 'G'] means
                        4 intervals: x < 'C', 'C' <= x < 'E', 'E' <= x < 'G', 'G' <= x)

    Return:
    result -- the partitioned subsets of D
    """
    result = []
    ### START CODE HERE ###

    # sort the dataset according to their values

    new_data = list(data)
    new_data.sort()

    # calculate the number of bins

    n_bins = len(range_indices)

    # for each bin

    for i in range(n_bins):
        # find elements belonging to each range

        s = [x for x in new_data if x[0] < range_indices[i]]

        # add the partitioned list to the result

        result.append(s)

        # find the last element in the previous partition

        last_element = s[-1] if s else None

        # find the index of the last element

        if last_element:
            last_index = new_data.index(last_element)

            # remove the partitioned list from the dataset

            new_data = new_data[last_index + 1 :]
        else:
            new_data = new_data
    # append the last remaining data list

    result.append([x for x in new_data if x[0] >= range_indices[n_bins - 1]])

    ### END CODE HERE ###

    return result

In [40]:
import multiprocessing as mp


def parallel_redistributed_groupby(dataset, range_indices):
    """
    Perform a parallel redistributed groupby method

    Arguments:
    dataset -- entire record set to be merged

    Return:
    result -- the aggregated record dictionary according to the group_by attribute index
    """

    result = []

    ### START CODE HERE ###

    # partition the dataset into 4 groups using range partitioning

    partitioned_data = range_partition(dataset, range_indices)

    # create with 4 processes

    pool = mp.Pool(processes=4)

    # perform local aggregation on each partition in parallel

    local_results = pool.map(local_groupby, partitioned_data)

    # close the pool

    pool.close()
    pool.join()

    # union the local results into the final result

    result = {k: v for d in local_results for k, v in d.items()}

    ### END CODE HERE ###

    return result

In [41]:
result = parallel_redistributed_groupby(D, ["C", "E", "G"])
print(result)

{'A': 10, 'B': 14, 'C': 18, 'D': 100, 'E': 140, 'F': 180, 'G': 140, 'H': 180}
